In [48]:
import math
import numpy as np
import pandas as pd
from dateutil.parser import parse

pd.set_option('display.max_columns', None)

In [49]:
def loadCSVFile(csvPath, sepStyle, encodeStyle):
    return pd.read_csv(csvPath, sep=sepStyle, encoding=encodeStyle, low_memory=False)

In [50]:
def mergeDataFrames(leftDF, rightDF, keyColumnL, keyColumnR, suffixTuple):
    return leftDF.merge(rightDF, how='left', left_on=keyColumnL, right_on=keyColumnR, suffixes=suffixTuple)

In [51]:
def mergeDtHrUTI(listDtInUTI, listHrInUTI, listDtOutUTI, listHrOutUTI):
    listDTHR_InUTI, listDTHR_OutUTI= [], []
    
    for tplDtHrInUTI in zip(listDtInUTI, listHrInUTI):
        inDtUTI = tplDtHrInUTI[0]
        inHrUTI = tplDtHrInUTI[1]
        if type(inDtUTI) != str and type(inHrUTI) != str:
            if math.isnan(inDtUTI) == True or math.isnan(inHrUTI) == True:
                listDTHR_InUTI.append(np.nan)
        else:
            listDTHR_InUTI.append(' '.join(tplDtHrInUTI))
            
    for tplDtHrOutUTI in zip(listDtOutUTI, listHrOutUTI):
        outDtUTI = tplDtHrOutUTI[0]
        outHrUTI = tplDtHrOutUTI[1]
        if type(outDtUTI) != str and type(outHrUTI) != str:
            if math.isnan(outDtUTI) == True or math.isnan(outHrUTI) == True:
                listDTHR_OutUTI.append(np.nan)
        else:
            listDTHR_OutUTI.append(' '.join(tplDtHrOutUTI))
            
    return listDTHR_InUTI, listDTHR_OutUTI

In [52]:
def timeCounterBtwnDates(bDate, eDate):
    _bDate = parse(bDate, dayfirst=True).ctime()
    _bDate = parse(_bDate, dayfirst=True)
    
    _eDate = parse(eDate, dayfirst=True).ctime()
    _eDate = parse(_eDate, dayfirst=True)

    delta = _eDate - _bDate

    totalDays = delta.total_seconds()/86400
    
    return totalDays

In [53]:
def getDaysCountByRow(row, posREG, posALTA, posInUTI, posOutUTI):
    if type(row[posREG]) == str and type(row[posInUTI]) == str and type(row[posOutUTI]) == str:
        Pre_UTITime = timeCounterBtwnDates(bDate=row[posREG], eDate=row[posInUTI])
        UTI_TIME = timeCounterBtwnDates(bDate=row[posInUTI], eDate=row[posOutUTI])
    
    if type(row[posREG]) == str and type(row[posALTA]) == str and type(row[posInUTI]) == float and type(row[posOutUTI]) == float:
        Pre_UTITime = timeCounterBtwnDates(bDate=row[posREG], eDate=row[posALTA])
        UTI_TIME = 0.0
        
    if type(row[posREG]) == float or type(row[posALTA]) == float:
        Pre_UTITime = np.nan
        UTI_TIME = np.nan
    
    if type(row[posREG]) == float and type(row[posALTA]) == str and type(row[posInUTI]) == float and type(row[posOutUTI]) == float:
        Pre_UTITime = np.nan
        UTI_TIME = np.nan
    
    return Pre_UTITime, UTI_TIME

In [54]:
def getTimeForEachDFRow(DFRows):
    list_PreUTITime, list_UTITime = [], []
    
    iREG=DF_REG_EST_UTI.columns.get_loc('DTHR_REGISTRO')
    iALTA=DF_REG_EST_UTI.columns.get_loc('DTHR_ALTA')
    iInUTI=DF_REG_EST_UTI.columns.get_loc('DTHR_IN_UTI')
    iOutUTI=DF_REG_EST_UTI.columns.get_loc('DTHR_OUT_UTI')
    for row in DFRows:
      pUTI, UTI = getDaysCountByRow(row=row, posREG=iREG, posALTA=iALTA, posInUTI=iInUTI, posOutUTI=iOutUTI)
      list_PreUTITime.append(pUTI)
      list_UTITime.append(UTI)
    
    return list_PreUTITime, list_UTITime

In [55]:
if __name__ == '__main__':
    
    path = './data/dataToPreProcessing/'
    
    # Carrega os dados
    DF_REGISTRO = loadCSVFile(csvPath=path+'atendimentos.csv', encodeStyle='latin1', sepStyle=';')
    DF_ESTUDO = loadCSVFile(csvPath=path+'estudo.csv', encodeStyle='latin1', sepStyle=';')
    DF_UTI = loadCSVFile(csvPath=path+'uti.csv', encodeStyle='latin1', sepStyle=';')
    
    
    #Remove duplicações
    DF_REGISTRO = DF_REGISTRO.drop_duplicates()
    DF_ESTUDO = DF_ESTUDO.drop_duplicates()
    DF_UTI = DF_UTI.drop_duplicates()

    #Corrige DF_UTI para que tenha uma entrada para cada passagem na UTI
    DF_UTI = DF_UTI.sort_values(by=['CD_ATENDIMENTO','DT_ENTRADA','HR_ENTRADA'],ignore_index=True)
    DF_UTI['REENTRADA']=0
    i = 0
    j = 0
    DF_UTI2 = DF_UTI.copy()
    while i < len(DF_UTI.index)-1 :
      if DF_UTI.loc[i,'CD_ATENDIMENTO'] == DF_UTI.loc[i+1,'CD_ATENDIMENTO'] :
        if type(DF_UTI.loc[i,'DT_SAIDA']) != str or type(DF_UTI.loc[i,'HR_SAIDA']) != str :
          print('PANIC!!!!!!!!!!!!')
          i=i+1
          j=i
        if timeCounterBtwnDates(bDate=DF_UTI.loc[i,'DT_SAIDA']+' '+DF_UTI.loc[i,'HR_SAIDA'], eDate=DF_UTI.loc[i+1,'DT_ENTRADA']+' '+DF_UTI.loc[i+1,'HR_ENTRADA']) < 1 :
#          print('merging two rows at: ',i)
          DF_UTI2.loc[j,'DT_SAIDA'] = DF_UTI.loc[i+1,'DT_SAIDA']
          DF_UTI2.loc[j,'HR_SAIDA'] = DF_UTI.loc[i+1,'HR_SAIDA']
          DF_UTI2 = DF_UTI2.drop([i+1])
          i = i+1
        else :
#          print('spliting UTI entrance at: ',i)
          DF_UTI2.loc[i+1,'REENTRADA']=DF_UTI2.loc[j,'REENTRADA']+1
          i = i+1
          j = i
      else :
        i = i+1
        j = i

    DF_UTI = DF_UTI2
    
    #União das Tabelas REGISTRO e ESTUDO
    DF_REG_EST = mergeDataFrames(leftDF=DF_REGISTRO,
                                 rightDF=DF_ESTUDO,
                                 keyColumnL='CD_PACIENTE', keyColumnR='CD_PACIENTE_FROM_ESTUDO',
                                 suffixTuple=('_FROM_REG', '_FROM_EST'))
    
    # It seems the datatypes of the CD_ATENDIMENTO in different tables are not the same
    DF_REG_EST['CD_ATENDIMENTO'] = pd.to_numeric(DF_REG_EST['CD_ATENDIMENTO'])
    DF_UTI['CD_ATENDIMENTO'] = pd.to_numeric(DF_UTI['CD_ATENDIMENTO'])
    # Also sort before merging with UTI so we can check if it's working more easily
    DF_REG_EST = DF_REG_EST.sort_values(by=['CD_ATENDIMENTO'],ignore_index=True)

    #União das Tabelas REGISTRO-ESTUDO e UTI
    DF_REG_EST_UTI = mergeDataFrames(leftDF=DF_REG_EST,
                                     rightDF=DF_UTI,
                                     keyColumnL='CD_ATENDIMENTO', keyColumnR='CD_ATENDIMENTO',
                                     suffixTuple=('_FROM_MRG_REG_EST', '_FROM_UTI'))
    
    
    #Remove duplicações
    DF_REG_EST_UTI = DF_REG_EST_UTI.drop_duplicates()
    
    
    #União dos campos DT_ENTRADA + HR_ENTRADA e DT_SAIDA + HR_SAIDA
    lDtInUTI = DF_REG_EST_UTI['DT_ENTRADA'].values.tolist()
    lHrInUTI = DF_REG_EST_UTI['HR_ENTRADA'].values.tolist()
    lDtOutUTI = DF_REG_EST_UTI['DT_SAIDA'].values.tolist()
    lHrOutUTI = DF_REG_EST_UTI['HR_SAIDA'].values.tolist()
    
    listDTHR_InUTI, listDTHR_OutUTI = mergeDtHrUTI(listDtInUTI=lDtInUTI,
                                                   listHrInUTI=lHrInUTI,
                                                   listDtOutUTI=lDtOutUTI,
                                                   listHrOutUTI=lHrOutUTI)
    
    #Adicionando União anterior a tabela geral
    DF_REG_EST_UTI['DTHR_IN_UTI'] = listDTHR_InUTI
    DF_REG_EST_UTI['DTHR_OUT_UTI'] = listDTHR_OutUTI
    
    #Calculando e Adicionando numero de dias ante e durante a UTI
    list_PreUTITime, list_UTITime = getTimeForEachDFRow(DFRows=DF_REG_EST_UTI.values.tolist())
    
    DF_REG_EST_UTI['DIAS_PRE_UTI'] = list_PreUTITime
    DF_REG_EST_UTI['DIAS_NA_UTI'] = list_UTITime


    print(DF_REG_EST['CD_ATENDIMENTO'].head(10))
    print(DF_UTI.head(10))
    print(DF_REG_EST_UTI['CD_ATENDIMENTO'].head(10),DF_REG_EST_UTI['REENTRADA'].head(10))


0     812107.0
1     925313.0
2    1219456.0
3    1219630.0
4    1219916.0
5    1219949.0
6    1221646.0
7    1223244.0
8    1225062.0
9    1225063.0
Name: CD_ATENDIMENTO, dtype: float64
   STATUS_COVID  CD_ATENDIMENTO  CD_PACIENTE_FROM_UTI  CD_LEITO  CD_UNIDADE  \
0   CONFIRMADOS          812107               1969172      1654           8   
1     NEGATIVOS          925313                454612      1605           7   
4     NEGATIVOS          925313                454612      1607           7   
5     NEGATIVOS          925313                454612      1613           7   
8     NEGATIVOS          925313                454612      1607           7   
9     NEGATIVOS          925313                454612      2736           5   
11    NEGATIVOS          925313                454612      1608           7   
12    NEGATIVOS         1223244               1204452      1608           7   
13    NEGATIVOS         1230987               1402382      1606           7   
14    NEGATIVOS        

In [56]:
#getDaysCountByRow(DF_REG_EST_UTI.values.tolist()[7], posREG=1, posALTA=27, posInUTI=119, posOutUTI=120)

In [57]:
#print('REG', DF_REG_EST_UTI.values.tolist()[7][1])

In [70]:
DF_REG_EST_UTI

,ID,DTHR_REGISTRO,PORTA_ENTRADA,DATA_INICIO_SINTOMAS,HISTORICO_VIAGEM,PAIS_VISITADO,CIDADES_VISITADAS,DATA_RETORNO,COLETOU_AMOSTRA,LOCAL_COLETA,DATA_COLETA,PCR_INFLUENZA,IFI_OUTRO_VIRUS,PCR_COVID_19,STATUS_COVID_FROM_MRG_REG_EST,STATUS_COVID_ORDEM,DESFECHO,CD_ATENDIMENTO,DS_TP_ATENDIMENTO,CD_PACIENTE,GENERO,IDADE,SN_OBITO,DIAGNOSTICO_SARS,OBITO_SARS,TIPO_SAIDA,TIPO_OBITO,DTHR_ALTA,CD_CID,DS_ESPECIALID_SAI,REINTER_30D,CARATER_ATENDIMENTO,EMERG_SERVICO,EMERG_RISCO,CID_DIAG_ATEND,CID_CAPITULO_DIAG,BLOCO_CENTRAL_ST,HEMODINAMICA_ST,EX_LAB_ST,EX_IMG_ST,EX_OUTROS_ST,CD_PACIENTE_FROM_ESTUDO,COMPETENCIA,PORTA_ENTRADA_FROM_ESTUDO,DATA_INICIO_SINTOMAS_FROM_ESTUDO,HISTORICO_VIAGEM_FROM_ESTUDO,CIDADES_VISITADAS_FROM_ESTUDO,LOCAL_COLETA_FROM_ESTUDO,DATA_COLETA_FROM_ESTUDO,PCR_COVID_19_FROM_ESTUDO,DESFECHO_FROM_ESTUDO,TIPO_REGISTRO,SN_IFI_OV_SINCICIAL_RESP,SN_IFI_OV_PARAINFLU3,SN_IFI_OV_PARAINFLU2,SN_IFI_OV_PARAINFLU1,SN_IFI_OV_INFLUA,SN_IFI_OV_INFLUB,SN_IFI_OV_ADENOVIRUS,SN_CRIT_CONTATO_CASO_COVID,SN_CRIT_CONTATO_CASO_SUSPEITO,SN_CRIT_PROF_SAUDE_HOSPITAL,SN_CRIT_PROF_SAUDE_OUT,SN_CRIT_SRAG,SN_CRIT_SINDROME_GRIPAL,SN_CRIT_IGNORADO,SN_CRIT_OUTROS,SN_CRIT_PROT_COVID19,ESTUDO_TP_EXPOSICAO,ESTUDO_COMORB_PREV_HAS,ESTUDO_COMORB_PREV_IAM,ESTUDO_COMORB_PREV_IC,ESTUDO_COMORB_PREV_DRC,ESTUDO_COMORB_PREV_DMI,ESTUDO_COMORB_PREV_DMII,ESTUDO_COMORB_PREV_CARCINOMA,ESTUDO_COMORB_PREV_ADENOMA,ESTUDO_COMORB_PREV_LINFOMA,ESTUDO_COMORB_PREV_AVC,ESTUDO_COMORB_PREV_ASMA,ESTUDO_COMORB_PREV_DPOC,ESTUDO_COMORB_PREV_OBESIDADE,ESTUDO_COMORB_PREV_EPILEPSIA,ESTUDO_COMORB_PREV_HIGIDO,ESTUDO_COMORB_PREV_OUTRAS,ESTUDO_CHARLSON,ESTUDO_FR_DIA_COLETA,ESTUDO_FC_DIA_COLETA,ESTUDO_PA_DIA_COLETA,ESTUDO_TEMPERATURA_DIA_COLETA,ESTUDO_SATURACAO_DIA_COLETA,ESTUDO_APACHE2,ESTUDO_RISCO,ESTUDO_TRATAMENTO,ESTUDO_COMORB_PREV_TX_ORG_SOL,ESTUDO_CONDUTA,ESTUDO_LOCAL_CONTAMINACAO,ESTUDO_LOCAL_CONTAM_UI,ESTUDO_DT_DESFECHO,ESTUDO_COMORB_PREV_DESCON,ESTUDO_COMORB_PREV_DVP,ESTUDO_COMORB_PREV_DEMENCIA,ESTUDO_COMORB_PREV_DTC,ESTUDO_COMORB_PREV_UP,ESTUDO_COMORB_PREV_DFL,ESTUDO_COMORB_PREV_DFM,ESTUDO_COMORB_PREV_HEMIPLEGIA,ESTUDO_COMORB_PREV_LEUCEMIA,ESTUDO_COMORB_PREV_HIV,ESTUDO_COMORB_PREV_TUL,ESTUDO_COMORB_PREV_TUM,STATUS_COVID_FROM_UTI,CD_PACIENTE_FROM_UTI,CD_LEITO,CD_UNIDADE,DS_UNIDADE,DT_ENTRADA,HR_ENTRADA,DT_SAIDA,HR_SAIDA,REENTRADA,DTHR_IN_UTI,DTHR_OUT_UTI,DIAS_PRE_UTI,DIAS_NA_UTI
0,7242,05/08/2020 11:37:00,NaN,NaN,NaN,NaN,NaN,NaN,SIM,7 S - INTERNACAO,05/08/2020 12:14:25,NaN,NaN,POSITIVO,CONFIRMADOS,1,NÃO INFORMADO,812107.0,INTERNADO,1969172.0,MASCULINO,"39,3641552511416",N,NÃO,NÃO,ALTA,NaN,28/02/2021 10:29:24,A499,CLINICA MEDICA,NÃO,URGÊNCIA,PRONTOSUS,VERMELHO,A499,Infecciosas e parasitárias,NÃO,NÃO,SIM,SIM,SIM,1969172.0,05/08/2020,NaN,NaN,NaN,NaN,7 S - INTERNACAO,05/08/2020 12:14:25,POSITIVO,INTERNAÇÃO,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,HOSPITALAR HSL,N,N,N,N,N,N,N,N,N,N,N,N,N,S,N,"EPILEPSIA, POLINEUROPATIA, ACAMADO CR¿NICO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,UNIDADE DE INTERNACAO HSL,13.0,NaN,N,N,N,N,N,N,N,N,N,N,N,N,CONFIRMADOS,1969172.0,1654.0,8.0,UTI,07/07/2019,01:16:30,17/07/2019,20:33:50,0.0,07/07/2019 01:16:30,17/07/2019 20:33:50,-395.430903,10.803704
1,4463,08/07/2020 12:56:20,NaN,NaN,NaN,NaN,NaN,NaN,SIM,9 N - INTERNACAO,08/07/2020,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,925313.0,INTERNADO,454612.0,MASCULINO,"42,498401826484",N,NÃO,NÃO,ALTA,NaN,07/10/2020 13:13:46,K263,NEUROCIRURGIA,NÃO,URGÊNCIA,PRONTOPUC,LARANJA,K263,Aparelho digestivo,SIM,SIM,SIM,SIM,SIM,454612.0,01/07/2020,NaN,NaN,NaN,NaN,9 N - INTERNACAO,08/07/2020,NEGATIVO,INTERNAÇÃO,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,HOSPITALAR HSL,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NEGATIVOS,454612.0,1605.0,7.0,UTI,02/10/2019,01:29:47,11/09/2019,22:32:21,0.0,02/10/2019 01:29:47,11/09/2019 22:32:21,-280.476771,-20.123218
2,4463,08/07/2020 12:56:20,NaN,NaN,NaN,NaN,NaN,NaN,SIM,9 N - INTERNACAO,08/07/2020,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,925313.0,INTERNA

In [69]:
listaColunasREG = DF_REG_EST_UTI.columns
print(listaColunasREG)

Index(['ID', 'DTHR_REGISTRO', 'PORTA_ENTRADA', 'DATA_INICIO_SINTOMAS',
       'HISTORICO_VIAGEM', 'PAIS_VISITADO', 'CIDADES_VISITADAS',
       'DATA_RETORNO', 'COLETOU_AMOSTRA', 'LOCAL_COLETA',
       ...
       'DS_UNIDADE', 'DT_ENTRADA', 'HR_ENTRADA', 'DT_SAIDA', 'HR_SAIDA',
       'REENTRADA', 'DTHR_IN_UTI', 'DTHR_OUT_UTI', 'DIAS_PRE_UTI',
       'DIAS_NA_UTI'],
      dtype='object', length=125)


In [59]:
#print('ALTA', DF_REG_EST_UTI.values.tolist()[7][27])

In [60]:
#print('InUTI', DF_REG_EST_UTI.values.tolist()[7][119])

In [61]:
#print('OutUTI', DF_REG_EST_UTI.values.tolist()[7][120])

In [62]:
DF_REG_EST_UTI.loc[DF_REG_EST_UTI['CD_ATENDIMENTO']==1584472.0]

,ID,DTHR_REGISTRO,PORTA_ENTRADA,DATA_INICIO_SINTOMAS,HISTORICO_VIAGEM,PAIS_VISITADO,CIDADES_VISITADAS,DATA_RETORNO,COLETOU_AMOSTRA,LOCAL_COLETA,DATA_COLETA,PCR_INFLUENZA,IFI_OUTRO_VIRUS,PCR_COVID_19,STATUS_COVID_FROM_MRG_REG_EST,STATUS_COVID_ORDEM,DESFECHO,CD_ATENDIMENTO,DS_TP_ATENDIMENTO,CD_PACIENTE,GENERO,IDADE,SN_OBITO,DIAGNOSTICO_SARS,OBITO_SARS,TIPO_SAIDA,TIPO_OBITO,DTHR_ALTA,CD_CID,DS_ESPECIALID_SAI,REINTER_30D,CARATER_ATENDIMENTO,EMERG_SERVICO,EMERG_RISCO,CID_DIAG_ATEND,CID_CAPITULO_DIAG,BLOCO_CENTRAL_ST,HEMODINAMICA_ST,EX_LAB_ST,EX_IMG_ST,EX_OUTROS_ST,CD_PACIENTE_FROM_ESTUDO,COMPETENCIA,PORTA_ENTRADA_FROM_ESTUDO,DATA_INICIO_SINTOMAS_FROM_ESTUDO,HISTORICO_VIAGEM_FROM_ESTUDO,CIDADES_VISITADAS_FROM_ESTUDO,LOCAL_COLETA_FROM_ESTUDO,DATA_COLETA_FROM_ESTUDO,PCR_COVID_19_FROM_ESTUDO,DESFECHO_FROM_ESTUDO,TIPO_REGISTRO,SN_IFI_OV_SINCICIAL_RESP,SN_IFI_OV_PARAINFLU3,SN_IFI_OV_PARAINFLU2,SN_IFI_OV_PARAINFLU1,SN_IFI_OV_INFLUA,SN_IFI_OV_INFLUB,SN_IFI_OV_ADENOVIRUS,SN_CRIT_CONTATO_CASO_COVID,SN_CRIT_CONTATO_CASO_SUSPEITO,SN_CRIT_PROF_SAUDE_HOSPITAL,SN_CRIT_PROF_SAUDE_OUT,SN_CRIT_SRAG,SN_CRIT_SINDROME_GRIPAL,SN_CRIT_IGNORADO,SN_CRIT_OUTROS,SN_CRIT_PROT_COVID19,ESTUDO_TP_EXPOSICAO,ESTUDO_COMORB_PREV_HAS,ESTUDO_COMORB_PREV_IAM,ESTUDO_COMORB_PREV_IC,ESTUDO_COMORB_PREV_DRC,ESTUDO_COMORB_PREV_DMI,ESTUDO_COMORB_PREV_DMII,ESTUDO_COMORB_PREV_CARCINOMA,ESTUDO_COMORB_PREV_ADENOMA,ESTUDO_COMORB_PREV_LINFOMA,ESTUDO_COMORB_PREV_AVC,ESTUDO_COMORB_PREV_ASMA,ESTUDO_COMORB_PREV_DPOC,ESTUDO_COMORB_PREV_OBESIDADE,ESTUDO_COMORB_PREV_EPILEPSIA,ESTUDO_COMORB_PREV_HIGIDO,ESTUDO_COMORB_PREV_OUTRAS,ESTUDO_CHARLSON,ESTUDO_FR_DIA_COLETA,ESTUDO_FC_DIA_COLETA,ESTUDO_PA_DIA_COLETA,ESTUDO_TEMPERATURA_DIA_COLETA,ESTUDO_SATURACAO_DIA_COLETA,ESTUDO_APACHE2,ESTUDO_RISCO,ESTUDO_TRATAMENTO,ESTUDO_COMORB_PREV_TX_ORG_SOL,ESTUDO_CONDUTA,ESTUDO_LOCAL_CONTAMINACAO,ESTUDO_LOCAL_CONTAM_UI,ESTUDO_DT_DESFECHO,ESTUDO_COMORB_PREV_DESCON,ESTUDO_COMORB_PREV_DVP,ESTUDO_COMORB_PREV_DEMENCIA,ESTUDO_COMORB_PREV_DTC,ESTUDO_COMORB_PREV_UP,ESTUDO_COMORB_PREV_DFL,ESTUDO_COMORB_PREV_DFM,ESTUDO_COMORB_PREV_HEMIPLEGIA,ESTUDO_COMORB_PREV_LEUCEMIA,ESTUDO_COMORB_PREV_HIV,ESTUDO_COMORB_PREV_TUL,ESTUDO_COMORB_PREV_TUM,STATUS_COVID_FROM_UTI,CD_PACIENTE_FROM_UTI,CD_LEITO,CD_UNIDADE,DS_UNIDADE,DT_ENTRADA,HR_ENTRADA,DT_SAIDA,HR_SAIDA,REENTRADA,DTHR_IN_UTI,DTHR_OUT_UTI,DIAS_PRE_UTI,DIAS_NA_UTI
24805,25975,08/03/2021 08:11:00,NaN,NaN,NaN,NaN,NaN,NaN,SIM,PRONTOPUC ADULTO/INTE,08/03/2021 08:28:39,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,1584472.0,INTERNADO,1832839.0,MASCULINO,"58,8792237442922",S,NÃO,NÃO,ÓBITO,NaN,21/03/2021 18:59:44,A310,CIRURGIA GERAL,NÃO,URGÊNCIA,PRONTOSUS,LARANJA,A419,Infecciosas e parasitárias,SIM,NÃO,SIM,SIM,SIM,1832839.0,08/03/2021,NaN,NaN,NaN,NaN,PRONTOPUC ADULTO/INTE,08/03/2021 08:28:39,NEGATIVO,INTERNAÇÃO,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NEGATIVOS,1832839.0,1649.0,8.0,UTI,08/03/2021,13:31:53,18/03/2021,23:22:48,0.0,08/03/2021 13:31:53,18/03/2021 23:22:48,0.222836,10.410359


In [71]:
#DF_REG_EST_UTI.loc[DF_REG_EST_UTI['CD_ATENDIMENTO']==1365570]
listaResumida = ["REENTRADA", "DIAS_PRE_UTI", "DIAS_NA_UTI", "CD_ATENDIMENTO", "CD_PACIENTE", "DTHR_REGISTRO", "DTHR_ALTA", "IDADE", "DESFECHO_FROM_ESTUDO"]

DF_ARE = DF_REG_EST_UTI[[*listaResumida]]
DF_ARE

,REENTRADA,DIAS_PRE_UTI,DIAS_NA_UTI,CD_ATENDIMENTO,CD_PACIENTE,DTHR_REGISTRO,DTHR_ALTA,IDADE,DESFECHO_FROM_ESTUDO
0,0.0,-395.430903,10.803704,812107.0,1969172.0,05/08/2020 11:37:00,28/02/2021 10:29:24,"39,3641552511416",INTERNAÇÃO
1,0.0,-280.476771,-20.123218,925313.0,454612.0,08/07/2020 12:56:20,07/10/2020 13:13:46,"42,498401826484",INTERNAÇÃO
2,1.0,-214.044688,11.081123,925313.0,454612.0,08/07/2020 12:56:20,07/10/2020 13:13:46,"42,498401826484",INTERNAÇÃO
3,2.0,-180.300764,-100.176007,925313.0,454612.0,08/07/2020 12:56:20,07/10/2020 13:13:46,"42,498401826484",INTERNAÇÃO
4,3.0,-267.806771,36.925845,925313.0,454612.0,08/07/2020 12:56:20,07/10/2020 13:13:46,"42,498401826484",INTERNAÇÃO
...,...,...,...,...,...,...,...,...,...
25787,NaN,NaN,NaN,1605909.0,1902403.0,30/03/2021 10:34:30,NaN,"88,67100456621",NaN
25788,NaN,NaN,NaN,NaN,NaN,07/07/2020 10:57:11,NaN,NaN,NaN
25789,NaN,NaN,NaN,NaN,NaN,21/08/2020 08:48:13,NaN,NaN,NaN
25790,NaN,NaN,NaN,NaN,NaN,15/09/2020 10:53:21,NaN,NaN,NaN


In [76]:
acward_reentry = [1365570, 1561739, 1309609, 1269701, 1321725, 1473220, 1568471, 1384977, 1307455, 1556953, 1332099, 1307951, 1467668, 1331083, 1327490, 1330701, 1316371, 1307418, 1424264, 1357191, 1316978, 1528772, 1344686, 1292539, 1282792, 1454288, 1356018, 1474852, 1547939, 1344096, 1325755, 1354719, 1455614, 1417470, 1297709, 1303784, 1305022, 1314636, 1392458, 1336253, 1342041, 1346883, 1358733, 1359259, 1422582, 1429283, 1433424, 1444935, 1474396, 1474735, 1483406, 1498942, 1500122, 1509446, 1532077, 1567762]

wagner_estudo = DF_ARE[DF_ARE['CD_ATENDIMENTO'].isin(acward_reentry)]

In [78]:
#wagner_estudo.to_csv('wagner_estudo.csv')

In [28]:
#getDaysCountByRow(DF_REG_EST_UTI.values.tolist()[2], posREG=1, posALTA=27, posInUTI=119, posOutUTI=120)

In [29]:
#print('REG', DF_REG_EST_UTI.values.tolist()[2][1])

In [30]:
#print('ALTA', DF_REG_EST_UTI.values.tolist()[2][27])

In [31]:
#print('InUTI', DF_REG_EST_UTI.values.tolist()[2][119])

In [32]:
#print('OutUTI', DF_REG_EST_UTI.values.tolist()[2][120])